In [9]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
import time
import json

In [10]:
options = webdriver.FirefoxOptions()
# options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

In [11]:
collection = 'https://xoxno.com/collection/CRWEAPONS-e5ab49'

In [12]:
data = {}

In [13]:
driver.get(collection)
time.sleep(10)

In [14]:
def foo(data, driver): 
    for item in driver.find_elements(By.XPATH, '/html/body/div[1]/div[1]/main/div/div[2]/div[4]/div/div/div/*'):
        if item.text.startswith('Liquid sell for'):
            continue
        item = item.find_element(By.TAG_NAME, "a")
        id = item.get_attribute("href").split('/')[-1]
        currency = item.find_element(By.XPATH, "./div/div/div[3]/div[1]/div/div/img").get_attribute("src").split('/')[-1].split('.')[0]
        amount = item.find_element(By.XPATH, "./div/div/div[3]/div[1]/div/div/span").text
        amount = float(amount.replace('K', ''))*1000 if 'K' in amount else float(amount)
        data[id] = {
            'price': {
                'currency': currency,
                'amount': amount
            }
        }

In [15]:
# scroll to the bottom
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    foo(data, driver)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)
    new_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)
    if new_height == last_height:
        foo(data, driver)
        break
    last_height = new_height

In [32]:
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [31]:
foo(data, driver)